In [1]:
import mss
import mss.tools
import time
import os
import glob



def flux_capture_temporaire(dossier_temp="temp_screens", intervalle=1.0):
# os.makedirs(dossier_temp, exist_ok=True) used to create the directory if it doesn't exist
    if not os.path.exists(dossier_temp):
        os.makedirs(dossier_temp)

    # --- INITIALISATION DE MSS (Le contexte doit englober la boucle) ---
    with (mss.mss() as sct):

        # 2. TROUVER LA FENÊTRE
        hwnd = win32gui.FindWindow(None, "BlueStacks App Player 2")
        if not hwnd:
            # Fallback : recherche partielle si le nom change
            def callback(h, handles):
                if "BlueStacks" in win32gui.GetWindowText(h):
                    handles.append(h)
            handles = []
            win32gui.EnumWindows(callback, handles)
            if handles:
                hwnd = handles[0]
            else:
                raise RuntimeError("BlueStacks window not found")

        while True:
            # Verification: If the window is minimised, we wait.
            if win32gui.IsIconic(hwnd):
                print("Window minimised, waiting...")
                time.sleep(1)
                continue

            # 3. MISE A JOUR DES COORDONNÉES DYNAMIQUES
            # On recalcule à chaque tour au cas où vous bougez la fenêtre
            rect = win32gui.GetWindowRect(hwnd)
            x = rect[0]
            y = rect[1] + 54  # Offset for the title bar
            w = rect[2] - x - 40
            h = rect[3] - y

            # Correction des bordures invisibles de Windows 10/11 (souvent -7 pixels)
            # if x is negatif but close to 0, we set it to 0 to avoid crash
            if x < 0 and x > -20: x = 0
            if y < 0 and y > -20: y = 0

            # Security check to prevent GetDIBits crash
            if w <= 0 or h <= 0:
                print("Invalid window dimensions, waiting...")
                time.sleep(1)
                continue

            monitor = {"top": y, "left": x, "width": w, "height": h}

            # 4. CAPTURE
            try:
                #timestamp used to avoid collisions by generating unique filenames
                timestamp = int(time.time())
                name_fichier = os.path.join(dossier_temp, f"capture_{timestamp}.png")

                sct_img = sct.grab(monitor)
                mss.tools.to_png(sct_img.rgb, sct_img.size, output=name_fichier)

                # Send the path to the main programme
                yield name_fichier

            except mss.exception.ScreenShotError as e:
                print(f"Capture error (the window may be off-screen) : {e}")

            # 5. NETTOYAGE


            time.sleep(intervalle)

# --- EXECUTION ---
print("Starting monitoring...")
try:
    for path_img in flux_capture_temporaire(intervalle=1.0):
        print(f"Traitement : {path_img}")
except KeyboardInterrupt:
    print("END")
except Exception as e:
    print(f"Fatal error : {e}")

Starting monitoring...
Fatal error : name 'win32gui' is not defined


In [2]:
<import mss
import time
import os
import glob
import  win32gui
def flux_capture_temporaire(dossier_temp="temp_screens", intervalle=1.0):
    """
    Générateur qui :
    1. Capture l'écran.
    2. Sauvegarde l'image.
    3. Yield (retourne) le chemin.
    4. Supprime les images plus vieilles que 'retention' (en secondes ou en nombre).
    """

    # Création du dossier temporaire s'il n'existe pas
    if not os.path.exists(dossier_temp):
        os.makedirs(dossier_temp)

    with mss.mss() as sct:
        # On définit quel écran capturer (monitor 1 par défaut)
        hwnd = win32gui.FindWindow(None, "BlueStacks App Player 2")
        if not hwnd:
           raise RuntimeError("BlueStacks window not found")


    # 3. RÉCUPÉRER LE RECTANGLE DE LA FENÊTRE (Window Rect)
    # Renvoie (left, top, right, bottom) par rapport au coin haut-gauche de l'écran (0,0)
    # C'est ce que Windows considère comme "la fenêtre entière".
    rect = win32gui.GetWindowRect(hwnd)

    x = rect[0]
    y = rect[1]
    w = rect[2] - x  # Largeur totale (Right - Left)
    h = rect[3] - y  # Hauteur totale (Bottom - Top)
    monitor = {"top": y, "left": x, "width": w, "height": h}


    while True:
            # 1. Génération du nom de fichier unique avec timestamp
            # timestamp en millisecondes pour éviter les collisions
            timestamp = int(time.time() * 1000)
            nom_fichier = os.path.join(dossier_temp, f"capture_{timestamp}.png")

            # 2. Capture et sauvegarde (sct.shot est simple mais grab+to_png est plus flexible)
            # Ici on utilise une méthode optimisée pour sauvegarder directement
            sct_img = sct.grab(monitor)
            mss.tools.to_png(sct_img.rgb, sct_img.size,output=nom_fichier)

            # 3. RETOURNE LE PATH (c'est ici que la magie opère)
            # Le script s'arrête ici et reprendra au prochain tour de boucle
            yield nom_fichier

            # 4. NETTOYAGE (Supprimer les images trop vieilles)
            # On liste tous les fichiers png du dossier
            fichiers = sorted(glob.glob(os.path.join(dossier_temp, "*.png")))

            # Si on a plus de fichiers que la limite de rétention, on supprime les plus vieux
            #while len(fichiers) > retention:
             #   vieux_fichier = fichiers.pop(0) # Prend le premier (le plus vieux)
                #try:
                    #os.remove(vieux_fichier)
                    # print(f"Nettoyage : {vieux_fichier} supprimé.")
                #except OSError:
                    #pass # Le fichier est peut-être déjà utilisé/supprimé

            # 5. Attente
            time.sleep(intervalle)

# --- COMMENT L'UTILISER DANS VOTRE PROGRAMME ---

print("Démarrage de la surveillance...")

# Cette boucle agit comme le "moteur"
for chemin_image in flux_capture_temporaire(dossier_temp="temp_screens",intervalle=1.0):

    # À ce stade, l'image existe sur le disque.
    # C'est ici que vous insérez votre logique (Envoi API, analyse ML, etc.)
    print(f"Nouvelle image disponible pour traitement : {chemin_image}")

    # Exemple de traitement simulé
    # features = extraire_features(chemin_image)

    # Note : Pas besoin de supprimer manuellement ici,
    # la fonction 'flux_capture_temporaire' s'occupe de supprimer
    # les images plus vieilles que 3 frames automatiquement au prochain tour.>

Démarrage de la surveillance...


ScreenShotError: gdi32.GetDIBits() failed.